<a href="https://colab.research.google.com/github/svvsaga/datascience_workshop/blob/martin/workshop_sesjon3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Workshop sesjon 3: Analyse med BigQuery


# Kort om BigQuery

Google BigQuery er en såkalt Online Analytics Processing serverless datavarehus løsning som støtter analyse av store datamengder (petabytes) uten å kreve at brukeren sette opp infrastruktur (servere) i bakgrunnen.

## Hva er OLAP?
OLAP er et akronym Online Analytical Processing. Med OLAP kan kan utføre multidimensjonelle analyser av forretningsdata, og OLAP datavarehus tilbyr kapabiliteter for komplekse beregninger, trendanalyser og sofistikert datamodellering. Det var noen i dataviterforum-kanalen som nevnte å prøve ut ClickHouse. ClickHouse er et OLAP datavarehus - og BigQuery er det samme. 

## Hvordan representeres data i et datavarehus?
I et datavarehus bor data i tabeller i et tabulært format med et såkalt skjema definert. Et skjema betyr at man har en predefinert type for hver kolonne av datene. Se for deg at dataene dine er representert på tabulært format, som i Excel

### Tabell Ordre
| KundeId | Ordrelinje | Pris   |
|---------|------------|--------|
|   1020  | 1          | 499    |
|   1020  | 2          | 299    |
|   1020  | 3          | 199    |
|   1020  | 4          | 499    |


For å hente ut summen av alle linjer i tabellen over, vil man i et datavarehus på samme måte som i Excel kunne bruke `SUM()` funksjonen for å gjøre det. I datavarehuset gjør man det med et spørrespråk som heter SQL.  BigQuery bruker [standard SQL](https://www.w3schools.com/sql/sql_intro.asp) som spørrespråk. Syntaks for spørrespråket i BigQuery kan man finne [her](https://cloud.google.com/bigquery/docs/reference/standard-sql/query-syntax).

## Hvorfor BigQuery?
Tidligere avarter av datavarehus - som Oracle ble typisk brukt av BI-analysemiljøer, men i den siste tiden har slike løsninger blitt mer og mer populære blant flere analytikermiljøer fordi datavarehusene har fått enda mer funksjonalitet innebygd, blant annet mulighet for å gjøre [GIS-analyser](https://cloud.google.com/bigquery/docs/reference/standard-sql/geography_functions) og [maskinlæring](https://cloud.google.com/bigquery-ml/docs/introduction) som lineærregresjon, nevrale nett, K-means clustering. En full oversikt over funksjoner i BigQuery finner man 

BigQuery er altså i bunn og grunn en database, bare at man slipper å forholde seg til hardware og infrastruktur i det hele tatt. og har innebygd massive mengder [funksjoner for forskjellige formål](https://cloud.google.com/bigquery/docs/reference/standard-sql/functions-and-operators).



## Denne sesjonen
I denne sesjonen skal vi gå gjennom noen av de fordelene man oppnå ved bruk av BigQuery og vise noen triks for å komme i gang med det.

Koden vi går gjennom kan både kjøres direkte i denne notebooken og  i [BigQuery UI gjennom nettleseren](https://console.cloud.google.com/bigquery). **Vi anbefaler å utforske/teste spørringer i BigQuery UI** der man få noen fordeler  som f.eks. en live query validator samt estimert bruk av datavolumen og kan utforske datastrukturen mer interaktivt.

Vi rekker ikke å komme innom alle konsepter, men forhåpentligvis trigger vi interessen tilstrekkelig til at dere kan utforske dette videre på egen hånd. BigQuery har mye dokumentasjon tilgjengelig og eksempler
* [Tutorials](https://cloud.google.com/bigquery/docs/tutorials)
* [Generell dokumentasjon]()
* [Beste praksiser](https://cloud.google.com/bigquery/docs/best-practices-performance-overview)

BigQuery har også en god del tilgjengelige data som man kan koble med egne datakilder for analyser.
* [BigQuery Public Datasets](https://console.cloud.google.com/marketplace/browse?filter=solution-type:dataset)
* [BigQuery Commercial Datasets](https://cloud.google.com/commercial-datasets)

Eksempelvis ligger alle data fra [OpenStreetMap der](https://console.cloud.google.com/marketplace/product/openstreetmap/geo-openstreetmap?project=project-5553349987741481369&folder=&organizationId=) og oppdateres kontinuerlig, så man kan eksempelvis fra det datasettet finne antallet eller geografien til alle bruer eller tuneller i Norge på en forholdsvis enkel måte.

## Først: Autentisering med colab notebook
For å kunne komme videre med eksemplene i notebooken, er det nødvendig å autentisere seg. Kjør cellen under.

In [2]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
project = "<prosjekt-navn i GCP>" # Endre denne

In [3]:
project = "saga-workshop-dtest-9hsr"

## Grunnlegende spørringer
BigQuery bruker standard SQL som språk. Cellen under viser en enkelt spørring som bare returnerer alle rader fra vår trafikkdata tabell. 
Ved bruk av en notebook er det viktig å huske begrensning på minne (~12GB i colab), slik at den veldig stor tabell kan sannsynligvis ikke lastes.
Her kan man benytte seg av en `LIMIT` clause som returnere bare X antall rader. 

In [ ]:
%%bigquery --project $project
SELECT
 trpId,
 total
FROM
  `saga-workshop-data-vu8x.workshop.timestrafikkdata`
LIMIT 1000

Kopier select statement inn til BigQuery UI og se på datamenge Query Validatoren angir. 
- Hvor mye data innegår i spørringen? 
- Er datamengden som prosesseres bakgrunnen avhengig avhengig av `LIMIT` clause?


BigQuery prises etter datamengenden som lagres eller analyseres. En spørring som analyserer mye data vil derfor koste mer.
Det finnes ulike måter å begrense datamengenden i en spørring. BigQuery behandler dataene kolonne-basert, dvs. jo flere kolonner man ta med i en spørring, desto mer kostnader oppstår.

Gå inn i BigQuery GUIet, kopier inn spørringene under og sammenlign hvor mye data de prosesserer hver. Prøv å begrense spørring nr. 2 ved a velge enkelte kolonner (f.eks. `trpId`).

```
SELECT
  *
FROM
  `saga-workshop-data-vu8x.workshop.timestrafikkdata` 
```

```
SELECT
  --<kolonne-navn>--
FROM
  `saga-workshop-data-vu8x.workshop.timestrafikkdata` 
```

- Hvor mye data innegår i spørringen? 
- Er datamengden avhengig av hvilke kolonne man spør mot?

I tillegg til standard data typer som f.eks FLOAT, INT og STRING har BigQuery også RECORDS (structs) og REPEATED kolonner (arrays). 

- RECORDS er sammensatt av én eller flere kolonner (et slags tabell inn i tabellen).  Det er mulig å ha en nesting av structs med flere nivåer. 
RECORDS typisk oppstå ved ingest av en nested JSON fil.
Kolonner i en RECORD kan refereres til med en `.` notasjon, f.eks. `total.volumeNumbers` 
- REPEATED kolonner inneholder arrays av en bestemt data type, f.eks. en array av FLOAT verdier. REPEATED kolloner kan brukes til å præ-aggregere data (én rad versus mange rader for å samle data).

Utforsk schema til trafikkdata tabellen. I BigQuery UI velg tabellen og trykk på `schema`. Hvilke 


### Aggregasjoner

Prøv å lage eg SUM() aggregasjon over total trafikkvolumen.

Hint: `volume` kolonne inneholder trafikkvolumet per måling og finnes under en nested RECORD:
  - total 
    - volumeNumbers 
      - volume

In [ ]:
%%bigquery --project $project 
SELECT
  --<kolonne-navn>--
FROM
  `saga-workshop-data-vu8x.workshop.timestrafikkdata`

Bruk SUM() aggregasjon sammen med en `group by` for å summere trafikkvolumen per trafikkregistreringspunkt `trpId` og sorter etter totalvolumen (fallende)


In [ ]:
%%bigquery --project $project 
SELECT
  --<kolonne-navn1>--,
  --<kolonne-navn2>--
FROM
  `saga-workshop-data-vu8x.workshop.timestrafikkdata`
GROUP BY
 --<kolonne-navn>--
ORDER BY
 --<kolonne-navn>--
DESC


#### Finne duplikater

Datasettene i BigQuery er vanligvis denormalisert og det er vanlig å finne noen duplikater enten pga duplikasjon i underliggende datakilder eller som et resultat av avvik i ingestpipelinen. En enkel måte å finne disse duplikatene er ved å beregne total antall rader per en key som skal være unik, her f.eks. en kombinasjon av en tellepunkt ID og tidspunkt av måling. I spørringen under brukes det `HAVING` til filtrering av resultatene etter spørringen er kjørt.

In [ ]:
%%bigquery --project $project 
SELECT
  trpId, `from`, `to`, COUNT(*) antall_rader
FROM
  `saga-workshop-data-vu8x.workshop.timestrafikkdata` 
GROUP BY 
  1,2,3
HAVING antall_rader > 1
ORDER BY 
  antall_rader 
DESC

Hvor mange rader har minst 1 duplikat?

Fjern duplikatene! 

Det finnes ulike muligheter å gjøre dette direkte i BigQuery. 
Hvis enkelte rader er **100%** lik er det mulig f.eks. å kjøre en enkelt `SELECT DISTINCT` spørring:

In [ ]:
%%bigquery --project $project 
WITH dummy AS (
  SELECT 
  "a" as id, 1 as value
  UNION ALL
  SELECT
  "a" as id, 1 as value -- duplicate
  UNION ALL
  SELECT
  "b" as id, 2 as value
)
SELECT DISTINCT * FROM dummy

## Analytiske funksjoner

I vårt tilfelle inneholder trafikkdata desverre rader der trafikkvolum har blitt oppdatert og vi har flere versjoner av den samme målingen der vi vil bare ta vare på den siste versjonen i vår arbeidstabell.

I slike situasjoner er det mulig å bruke en [Analytisk funksjon](https://cloud.google.com/bigquery/docs/reference/standard-sql/analytic-function-concepts). Analytiske funksjoner kan brukes til å utføre aggregasjoner per gruppe lignende som group by, men resultatet returnerer **én rad per hver input rad**. 

Her kan vi bruke analytisk funksjon `ROW_NUMBER()` til å nummerere radene gruppert etter registereringspunkt (trpId), start (from)- og slutttidspunk  (to)og sortert fallende (`DESC`) etter data importeringstidspunkt. Til slutt bruker spørringen en `WHERE` clause for å velge den første raden per gruppe.

In [ ]:
%%bigquery --project $project 
SELECT
  trpId,
  `from`,
  `to`,
  ROW_NUMBER() OVER(PARTITION BY trpId ORDER BY ingest_time DESC) AS rn
FROM
  `saga-workshop-data-vu8x.workshop.timestrafikkdata`
WHERE 
  RN = 1

Spørsmål:
- Hva skjer når man fjerner WHERE clause?
- Hvordan ville man fått ut dataene til den *første* importeringstidspunkt (per hver gruppe)?

### Joins
BigQuery støtter alle vanlige typer joins over tabeller som left/right join, inner join, full outer join, cross join. En join benyttes når man ønsker å koble flere tabeller sammen. Dokumentasjon med eksempler finnes [her](https://cloud.google.com/bigquery/docs/reference/standard-sql/query-syntax#join_types)

Lag en left outer join, som knytter sammen et utvalg av kolonner i trafikkdata med lokasjonsinformasjon om trafikkregistreringspunkter. Bruk `trpId` fra trafikkdata tabellen og `id` fra registreringspuntktabellen i JOIN statement.

In [4]:
%%bigquery --project $project 
SELECT
  a.trpId,
  a.from,
  a.to,
  a.total.volumeNumbers.volume,
  b.location.coordinates.latLon
FROM
  `saga-workshop-data-vu8x.workshop.timestrafikkdata` a
LEFT JOIN
  `saga-workshop-data-vu8x.workshop.trafikkregistreringspunkter` b
ON
  -- join statement --
  a.trpId = b.id
LIMIT 1000

,trpId,from,to,volume,latLon
0,33902V320184,2019-08-16T20:00+02:00[Europe/Oslo],2019-08-16T21:00+02:00[Europe/Oslo],292,"{'lon': 5.733602, 'lat': 58.935201}"
1,33902V320184,2019-01-26T16:00+01:00[Europe/Oslo],2019-01-26T17:00+01:00[Europe/Oslo],569,"{'lon': 5.733602, 'lat': 58.935201}"
2,33902V320184,2019-03-23T18:00+01:00[Europe/Oslo],2019-03-23T19:00+01:00[Europe/Oslo],507,"{'lon': 5.733602, 'lat': 58.935201}"
3,33902V320184,2019-01-26T15:00+01:00[Europe/Oslo],2019-01-26T16:00+01:00[Europe/Oslo],693,"{'lon': 5.733602, 'lat': 58.935201}"
4,81921V2407139,2016-03-09T15:00+01:00[Europe/Oslo],2016-03-09T16:00+01:00[Europe/Oslo],261,"{'lon': 7.78701, 'lat': 62.975648}"
...,...,...,...,...,...
995,47254V121508,2021-03-29T08:00+02:00[Europe/Oslo],2021-03-29T09:00+02:00[Europe/Oslo],497,"{'lon': 7.994621, 'lat': 58.148466}"
996,47254V121508,2021-03-29T09:00+02:00[Europe/Oslo],2021-03-29T10:00+02:00[Europe/Oslo],512,"{'lon': 7.994621, 'lat': 58.148466}"
997,04925V444232,2021-03-29T06:00+02:00[Europe/Oslo],2021-03-29T07:00+02:00[Europe/Oslo],2013,"{'lon': 10.633088, 'lat': 59.911891}"
998,04925V444232,2021-03-29T06:00+02:00[Europe/Oslo],2021-03-29T07:00+02:00[Europe/Oslo],2013,"{'lon': 10.633088, 'lat': 59.911891}"


### Views
Views er en enkel måte å lagre en spørringen der resultatene oppdateres med endringer i underliggende datakilder. 

Bruk JOIN-spørringen fra før som view definisjon under og lagre viewen i ditt  workshop prosjekt. Bruk følgende skjelettet:


In [ ]:
%%bigquery --project $project 
CREATE VIEW workshop.trafikkdata_view AS (
-- view definition -- 
)

Kjør en spørring mot viewet:

In [ ]:
%%bigquery --project $project 
SELECT 
 * 
FROM workshop.trafikkdata_view
LIMIT 100

## Partisjonerte tabeller
BigQuery's forbruk av data (analysert datamengde) avhenger i stor grad av antall/størrelse av kolonner en spørring bruker. 

En `WHERE` clause hjelper generelt ikke med begrensningen av datamengden siden den tar effekt etter dataene er hentet.
Ett untak er spørring mot [partisjonerte tabeller](https://cloud.google.com/bigquery/docs/partitioned-tables), der en `WHERE` clause effektivt begrenser datamengden ved å velge partisjoner.

Vanligvis bruker vi tidspartisjonerte tabeller.
Tidspartisjonerte tabeller krever minst en kolonne som inneholder TIMESTAMP, DATETIME eller DATE verdier. I trafikkdata tabellen har vi flere slike kolonner
som angi både en tidspunkt av målingen (`from`, `to`) og en tidspunkt der dataene blir hentet inn i vårt system (`ingest_time`).



I cellen under oppretter vi en tidspartisjonert tabell.
- Først konverterer vi `from` kolonnen fra STRING til TIMESTAMP data type
- Så bruker vi en CREATE TABLE statement til å lage en ny tabell med tidspartisjonering på konverterte `from` kolonnen

In [ ]:
%%bigquery --project $project 
/* 
Example how to parse timestamps from string
*/
WITH original AS (
  SELECT
    "2016-06-22T19:00+02:00[Europe/Oslo]" AS date_string,
)
SELECT
  date_string, 
  REGEXP_EXTRACT(date_string, r'\[([^\[\]]*)\]') timezone, 
  DATETIME(PARSE_TIMESTAMP("%Y-%m-%dT%H:%M%Ez %Z ", REGEXP_REPLACE(date_string, r'[\[\]]', ' '))) AS datetime_utc,
  DATETIME(PARSE_TIMESTAMP("%Y-%m-%dT%H:%M%Ez %Z ", REGEXP_REPLACE(date_string, r'[\[\]]', ' ')), REGEXP_EXTRACT(date_string, r'\[([^\[\]]*)\]') ) AS datetime_oslo
from original

Lag en tabell som er partisjonert på en DATETIME/DATE kolonne. 

In [ ]:
%%bigquery --project $project 
CREATE TABLE workshop.trafikkdata_sub_partitioned PARTITION BY DATE(datetime) 
AS (
SELECT
  a.trpId,
  -- <datetime transformation> datetime 
  a.from,
  a.to,
  a.total.volumeNumbers.volume,
  b.location.coordinates.latLon
FROM
  `saga-workshop-data-vu8x.workshop.timestrafikkdata` a
LEFT JOIN
  `saga-workshop-data-vu8x.workshop.trafikkregistreringspunkter` b
ON
  a.trpId = b.Id
)

Sammenligne datamengden ved prosessering av følgende to spørringer i BigQuery GUIet:

```
SELECT 
 * 
FROM 
 workshop.trafikkdata_sub_partitioned` 
WHERE 
 DATE(datetime) = "2019-11-01"
```


```
SELECT 
 * 
FROM 
 `workshop.trafikkdata_sub_partitioned`
```

**NB:** I tabellen `timestrafikkdata_v2` har vi transformet `from` kolonnen til en kolonne med `TIMESTAMP` elementer og gitt den navnet `startOfHour`.

## GIS funksjoner
Støtte for en del GIS-funksjoner gjør BigQuery også til et kraftig analyseverktøy innen geoprosessering. BigQuery bruker en egen `GEOGRAPHY` data type til å håndtere objekter med geolokasjoner og tilbyr ulike funksjoner som hjelper med å transformere andre data typer til dette.

* [Intro til BigQuery GIS](https://cloud.google.com/bigquery/docs/gis-getting-started)
* [Oversikt over støttede GIS-funksjoner](https://cloud.google.com/bigquery/docs/gis-getting-started)

`GEOGRAPHY` datatypen beskriver et `punktsett` på jordas overflate. Et punktsett er et sett med punkter, linjer og polygoner på WGS84 referansesferoiden med geodetiske kanter. Litt beskrivelser av dette finner man [her](https://mentin.medium.com/long-lines-b65ad9fa8e14) 

BigQuery GIS støtter ikke følgende features i geospatial formater:

* Three-dimensional geometries. This includes the "Z" suffix in the WKT format, and the altitude coordinate in the GeoJSON format.
* Linear reference systems. This includes the "M" suffix in WKT format.
* WKT geometry objects other than geometry primitives or multipart geometries. In particular, BigQuery GIS supports only Point, MultiPoint, LineString, MultiLineString, Polygon, MultiPolygon, and GeometryCollection.

Et geografisk punkt kan lages med ST_GEOGPOINT() funksjon ved bruk av GPS lengdregrad og breddegrad. ST_GEOGPOINT() krever at lengdegrad og breddegrad er i FLOAT64 data type. Det er mulig å bruke `CAST(<string_kolonne> AS FLOAT64) <ny_kolonnenavn>`

In [ ]:
%%bigquery --project $project
SELECT 
  id,
  ST_GEOGPOINT(--<velg lengdegrad kolonne> , <velg breddegrad kolonne>--) geo
FROM 
  `saga-workshop-data-vu8x.workshop.trafikkregistreringspunkter`

Visualiser resultatet av spørringene i BigQuery GeoViz: https://bigquerygeoviz.appspot.com/ 
- I GeoViz, autoriser sesjon med GCP brukeren
- Lim in spørringen fra forigge celle
- Klikk "kjør"
- Under "Data" velg kolonnen som inneholder GEOGRAPHY data type 
- Ev. endre på "Style" (f.eks. set en verdi for "Circle radius") 

Velg et registreringspunkt og finn alle registreringspunkter som er i nærheten (avstand < 10km)

In [ ]:
%%bigquery --project $project
WITH
  point AS (
  SELECT
    ST_GEOGPOINT(location.coordinates.latLon.lon,
      location.coordinates.latLon.lat) geo
  FROM
    `saga-workshop-data-vu8x.workshop.trafikkregistreringspunkter`
  LIMIT
    1
),
all_points AS (
  SELECT
    *,
    ST_GEOGPOINT(location.coordinates.latLon.lon,
      location.coordinates.latLon.lat) geo
  FROM
    `saga-workshop-data-vu8x.workshop.trafikkregistreringspunkter`
)

SELECT
  all_points.geo,
  ST_DISTANCE(point.geo,
    all_points.geo) distance_in_meters
FROM
  point,
  all_points
WHERE
  ST_DWITHIN(point.geo,
    all_points.geo,
    10000)

Bruk en join, aggregasjon og GIS funksjonalitet sammen til å visualisere total trafikkvolum per tellepunkt i 2019 i et GeoViz kart.
Modifiser følgende spørring (husk at i tabellen timestrafikkdata_v2 er `from` kolonnen transformert til en kolonne med navnet `startOfHour` som inneholder `TIMESTAMP` elementer):

In [ ]:
%%bigquery --project $project
SELECT
  b.Id,
  SUM(--<velg trafikkvolumen kolonne>--) totalVolume,
  ANY_VALUE(ST_GEOGPOINT(--<velg lengdegrad kolonne> , <velg breddegrad kolonne>--)) geo
FROM
  `saga-workshop-data-vu8x.workshop.timestrafikkdata_v2` a
LEFT JOIN
  `saga-workshop-data-vu8x.workshop.trafikkregistreringspunkter` b
ON  
  a.trpId = b.Id
WHERE 
  -- <DATETIME filter for 2019> -- 
GROUP BY 1

- Skaler "Circle radius" etter total trafikk (`totalVolume`)

BigQuery lar oss også se på romlige forhold mellom geomerties. Under vises en spørring som viser hvordan
- `ST_WITHIN` kan brukes for å se om en geometry inneholder en annen 
- `ST_INTERSECTS` kan brukes for å se om en geometry krysser en annen

Illustrasjonen under viser hvordan de ulike geometriene ligger i forhold til hverandre.

<img src="https://drive.google.com/uc?id=1zTHJgFFWyDG_xbjjdmf-OEoKxu9-vmS5" width="500" height="500">

In [ ]:
%%bigquery --project $project
WITH polygon as (
    SELECT ST_GEOGFROMTEXT("POLYGON((0 0, 0 1, 1 1, 1 0, 0 0))") poly
),
geos as (
    SELECT ST_GEOGFROMTEXT("POINT(1.25 1.25)") dot, ST_GEOGFROMTEXT("LINESTRING(-0.5 -0.5, 0.5 0.5)") line
    UNION ALL 
    SELECT ST_GEOGFROMTEXT("POINT(0.75 0.75)") dot, ST_GEOGFROMTEXT("LINESTRING(-0.5 -0.5, -0.5 1.5)") line
)

SELECT *, ST_WITHIN(dot, poly) st_within, ST_INTERSECTS(line, poly) st_intersects
FROM polygon, geos

Aggreger trafikkvolum per kommune eller fylke ved bruk av geografisk intersects. Her benytter vi oss av geografisk definisjon av kommuner (POLYGON type) vi har lastet opp i forrige sesjon. 


Kjør en spørring for å hente ut alle registreringspunkter som overlapper med et gitt området.  Bruk f.eks. ST_WITHIN() funksjon. 

In [ ]:
%%bigquery --project $project
SELECT
  b.kommunenavn,
  COUNT(DISTINCT a.id) registreringspunkter,
  AVG(c.total.volumeNumbers.volume) avg_volume,
  ANY_VALUE(b.geometry) kommune_geografi
FROM
  `saga-workshop-data-vu8x.workshop.trafikkregistreringspunkter` a
JOIN
  `saga-workshop-data-vu8x.workshop.kommuner` b
ON
  ST_WITHIN( ST_GEOGPOINT(location.coordinates.latLon.lon,
      location.coordinates.latLon.lat),
    b.geometry)
LEFT JOIN
  `saga-workshop-data-vu8x.workshop.timestrafikkdata` c
ON
  a.id = c.trpId
GROUP BY
  1

NB: Rekkefølgen av JOINS er viktig mtp. performance! En JOIN basert på en geografisk sammenligning, som ST_WITHIN(), krever ofte mer prosesseringstid og burde derfor begrenses.

Visualiser totalttrafikk per området ved bruk av BigQuery GeoViz: https://bigquerygeoviz.appspot.com/  

## Brukerfunksjoner (UDFs) - Kjør denne som demo
Av og til mangles det en enkel funksjon som er ikke definert i standard SQL eller i BigQuery's ekstra funksjoner. Det er enkelt å definere og bruke egne funksjoner (UDFs) enten temporær i en skript eller lagre dem globalt i et prosjekt.  UDFs tilbyr også en interface til javascript kode/biblioteker, som gjør det mulig å integrere ekstra funksjonalitet.

In [ ]:
%%bigquery --project $project
/*
Todo: Definer ST_Buffer funksjon her
*/

Bruk ST_Buffer funksjon og visualiser trafikkregistreringspunktene (GeoViz).
